# Machine Learning with Linear Regression !

* It was derived from a survey of women in 1974 by Redbook magazine, in which married women were asked about their participation in extramarital affairs.

* Description of Variables !

* The dataset contains 6366 observations of 10 variables:(modified and cleaned)

* Now, perform Classification using logistic regression and check your model accuracy using confusion matrix and also through .score() function.

* NOTE: Perform OneHotEncoding for occupation and occupation_husb, since they should be treated as categorical variables. Careful from dummy variable trap for both!!

* Predict the probability of an affair for a random woman not present in the dataset. She's a 25-year-old teacher who graduated college, has been marriedfor 3 years, has 1 child, rates herself as strongly religious, rates her marriage as fair, and her husband is a farmer.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# reading the data from csv !

dataset = pd.read_csv('affairs.csv')

In [3]:
# lets do some investigation about the dataset !

dataset.shape

(6366, 9)

In [4]:
dataset.columns

Index(['rate_marriage', 'age', 'yrs_married', 'children', 'religious', 'educ',
       'occupation', 'occupation_husb', 'affair'],
      dtype='object')

In [5]:
dataset.columns.tolist()

['rate_marriage',
 'age',
 'yrs_married',
 'children',
 'religious',
 'educ',
 'occupation',
 'occupation_husb',
 'affair']

In [6]:
dataset.describe()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair
count,6366.000000,6366.000000,6366.000000,6366.000000,6366.000000,6366.000000,6366.000000,6366.000000,6366.000000
mean,4.109645,29.082862,9.009425,1.396874,2.426170,14.209865,3.424128,3.850141,0.322495
std,0.961430,6.847882,7.280120,1.433471,0.878369,2.178003,0.942399,1.346435,0.467468
min,1.000000,17.500000,0.500000,0.000000,1.000000,9.000000,1.000000,1.000000,0.000000
25%,4.000000,22.000000,2.500000,0.000000,2.000000,12.000000,3.000000,3.000000,0.000000
50%,4.000000,27.000000,6.000000,1.000000,2.000000,14.000000,3.000000,4.000000,0.000000
75%,5.000000,32.000000,16.500000,2.000000,3.000000,16.000000,4.000000,5.000000,1.000000
max,5.000000,42.000000,23.000000,5.500000,4.000000,20.000000,6.000000,6.000000,1.000000


In [7]:
dataset.sample(5)

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair
1183,3,22.0,2.5,0.0,3,14,3,4,1
2599,5,32.0,16.5,2.0,1,12,2,3,0
2274,5,17.5,2.5,0.0,3,12,3,2,0
4309,5,32.0,16.5,3.0,3,12,2,2,0
1302,3,32.0,13.0,1.0,1,16,4,2,1


In [8]:
dataset.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair
0,3,32.0,9.0,3.0,3,17,2,5,1
1,3,27.0,13.0,3.0,1,14,3,4,1
2,4,22.0,2.5,0.0,1,16,3,5,1
3,4,37.0,16.5,4.0,3,16,5,5,1
4,5,27.0,9.0,1.0,1,14,3,4,1


In [9]:
dataset.isnull().any(axis = 0)

rate_marriage      False
age                False
yrs_married        False
children           False
religious          False
educ               False
occupation         False
occupation_husb    False
affair             False
dtype: bool

In [10]:
dataset.dtypes # -> checking is there any categorical value exists in our dataset !

rate_marriage        int64
age                float64
yrs_married        float64
children           float64
religious            int64
educ                 int64
occupation           int64
occupation_husb      int64
affair               int64
dtype: object

In [11]:
#separating out data into Dependent(features) and independent(labels) variables !

fe = dataset.iloc[: , :-1].values
la = dataset.iloc[: , -1].values

In [12]:
fe

array([[ 3. , 32. ,  9. , ..., 17. ,  2. ,  5. ],
       [ 3. , 27. , 13. , ..., 14. ,  3. ,  4. ],
       [ 4. , 22. ,  2.5, ..., 16. ,  3. ,  5. ],
       ...,
       [ 5. , 22. ,  2.5, ..., 14. ,  3. ,  1. ],
       [ 5. , 32. ,  6. , ..., 14. ,  3. ,  4. ],
       [ 4. , 22. ,  2.5, ..., 16. ,  2. ,  4. ]])

In [13]:
la

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [14]:
type(fe)

numpy.ndarray

In [15]:
type(la)

numpy.ndarray

In [16]:
def Model(features, labels):
    # Applying OneHotEncoding !
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import OneHotEncoder
    
    col_to_ohe = [6,7]  # Columns to be OneHotEncoded !
    ohe = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [6,7])], remainder='passthrough')

    features = np.array(ohe.fit_transform(features))

    
    # Getting indexes for the columns to be dropped, to avoid dummy variable trap !
    total_col, indexes = 0, []
    for col in col_to_ohe:
        unique_val_count = len(dataset.iloc[:,col].value_counts())
        total_col += unique_val_count
        indexes.append(total_col - unique_val_count)
    
    # Dropping the dummy variable trap columns !
    features = np.delete(features, indexes, axis=1)
    
    # Splitting the dataset into train and test !
    from sklearn.model_selection import train_test_split as TTS
    
    f_train,f_test,l_train,l_test = TTS(features, labels, test_size = 0.25,
                                        random_state = 0)
    
    # Logistic Regression Model !
    from sklearn.linear_model import LogisticRegression
    reg = LogisticRegression(random_state=0)
    reg = reg.fit(f_train, l_train)
    
    pred = reg.predict(f_test)   # Prediction on test data !
    
    # np.array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 25, 3, 1, 4, 16]).reshape(1,-1)
    # Preprocessing the new individual's data !
    val = np.array([3, 25, 3, 1, 4, 16, 4, 2]).reshape(1,-1)
    val = np.array(ohe.transform(val))
    val = np.delete(val, indexes, axis=1)
    
    val_pred = reg.predict_proba(val)  # Predicting Individual's value !
    
    # Confusion Matrix !
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(l_test, pred)
    
    # check the accuracy on the Model !
    mod_score = reg.score(f_test, l_test)
    
    return pred,val_pred,cm,mod_score

Pred, val_Pred, CM, Score = Model(fe,la)

print ("model accuracy using confusion matrix : "+str(CM))
print ("model accuracy using .score() function : "+str(round(Score*100,2)))
print ("percentage of total women actually had an affair : "+str(round(dataset["affair"].mean()*100,2))+"%")
print ("probability of an affair for a random woman is : "+str(val_Pred))


model accuracy using confusion matrix : [[993 107]
 [316 176]]
model accuracy using .score() function : 73.43
percentage of total women actually had an affair : 32.25%
probability of an affair for a random woman is : [[0.77223839 0.22776161]]


C:\Users\Bad Boy\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
